In [1]:
from ultralytics import YOLO
import multiprocessing

In [5]:
player_model = YOLO("models/yolo/yolov8t.pt")
court_model = YOLO("models/yolo/yolov8c.pt")

In [6]:
player_detections = player_model.track(source="data/yolo/video.mp4", project="./", conf=0.1, stream=True, show=True)
court_detections = court_model.track(source="data/yolo/video.mp4", project="./", conf=0.1, stream=True, show=True)

In [7]:
def detect(detection_list, detections):
    for detection in detections:
         detection_list.append(detection)

In [8]:
def track():
    thread_manager = multiprocessing.Manager()
    player_detection_list = thread_manager.list()
    court_detection_list = thread_manager.list()

    player_thread = multiprocessing.Process(target=detect, args=(player_detection_list, player_detections))
    court_thread = multiprocessing.Process(target=detect, args=(court_detection_list, court_detections))

    player_thread.start()
    court_thread.start()

    index = 0
    while player_thread.is_alive() or court_thread.is_alive() or player_detection_list or court_detection_list:
        if len(player_detection_list) <= index and len(court_detection_list) <= index:
            continue

        player_data = player_detection_list[index] 
        court_data = court_detection_list[index]
        index += 1
        
    player_thread.join()
    court_thread.join()


In [9]:
track()

TypeError: cannot pickle 'generator' object